In [ ]:
#----------------------------------------------
#This notebook further sets up data for creating visualizations and for performing analysis
#----------------------------------------------
%matplotlib notebook
import pandas as pd
import numpy as np
import os as os
import csv
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import median_test

In [ ]:
#Read and store the merged clean data and population data
event_data = os.path.join('..','Cleaned Data','Merged_Clean_Data.csv')
pop_data = os.path.join('..','Cleaned Data','Zillow_Population_Return.csv')

event_df = pd.read_csv(event_data)
pop_df = pd.read_csv(pop_data)

event_df = event_df.drop(event_df.columns[0], axis = 1)
pop_df = pop_df.drop(pop_df.columns[0], axis = 1)

In [ ]:
#create empty list for normalized event and population time series data
n = 6
event_lists = [[] for i in range(n)]
pop_lists = [[] for j in range(n)]


#create dictionary with data columns desired prior to adding normalized time series data
normalized_dict = {'title' : event_df['Title'], 'disaster number' : event_df['Disaster Number'],
                          'DisasterType' : event_df['Incident Type'], 'Declaration Date' : event_df['Declaration Date'],
                          'Zip Code' : event_df['RegionName'], 'County' : event_df['CountyName']}

#loop through empty event lists and then loop through event dataframe to populate empty event lists using declaration date to determine column index
event_list_count = -1
for x in event_lists:
    event_list_count += 1
    for index, row in event_df.iterrows():
        ddate = row['Declaration Date']
        try:
            col_idx = event_df.columns.get_loc(ddate)
            event_lists[event_list_count].append(row.iloc[col_idx + event_list_count + 1])
        except:
            event_lists[event_list_count].append(np.nan)

#loop through empty population lists and then loop through population dataframe to populate empty event lists using declaration date from event dataframe to determine column index
pop_list_count = -1
for y in pop_lists:
    pop_list_count += 1 
    for index, row in event_df.iterrows():
        ddate = row['Declaration Date']
        try:
            col_idx = pop_df.columns.get_loc(ddate)
            pop_lists[pop_list_count].append(pop_df.iloc[0, col_idx + pop_list_count + 1])
        except:
            pop_lists[pop_list_count].append(np.nan)

#loop through populated event lists and add key, value pairs to normalized_dict
event_key_count = -1
for a in event_lists:
    event_key_count += 1
    normalized_dict.update({f't+{event_key_count + 1}': event_lists[event_key_count]})

#loop through populated population lists and add key, value pairs to normalized_dict
pop_key_count = -1
for b in pop_lists:
    pop_key_count += 1
    normalized_dict.update({f't+{pop_key_count + 1}p': pop_lists[pop_key_count]})

#create data frame from normalized_dict
normalized_df = pd.DataFrame(normalized_dict)

#clean normalized_df of rows with NaN
normalized_clean_df = normalized_df.dropna()

normalized_clean_df.head()

In [ ]:
normalized_clean_df['DisasterType'].value_counts()

In [ ]:
normalized_clean_df = normalized_clean_df.set_index(['DisasterType', 'Zip Code'])

In [ ]:
normalized_clean_df.to_csv('../Cleaned Data/Normalized_Clean_Data.csv')

In [ ]:
#create individual dataframes for each disaster type to chart later
flood_df = normalized_clean_df.loc['Flood', :]
flood_df.to_csv('../Cleaned Data/Flood_Data.csv')


hurricane_df = normalized_clean_df.loc['Hurricane', :]
hurricane_df.to_csv('../Cleaned Data/Hurricane_Data.csv')


fire_df = normalized_clean_df.loc['Fire', :]
fire_df.to_csv('../Cleaned Data/Fire_Data.csv')


storms_df = normalized_clean_df.loc['Severe Storm(s)', :]
storms_df.to_csv('../Cleaned Data/Storms_Data.csv')


snow_df = normalized_clean_df.loc['Snow', :]
snow_df.to_csv('../Cleaned Data/Snow_Data.csv')


ice_df = normalized_clean_df.loc['Severe Ice Storm', :]
ice_df.to_csv('../Cleaned Data/Ice_Data.csv')

In [ ]:
#Test for the first Month
housing = flood_df['t+1']
pop = flood_df['t+1p']

stats.median_test(housing, pop)

In [ ]:
#Test for the Second Month
housing = flood_df['t+2']
pop = flood_df['t+2p']

stats.median_test(housing, pop)

In [ ]:
#Test for the third Month
housing = flood_df['t+3']
pop = flood_df['t+3p']

stats.median_test(housing, pop)

In [ ]:
#Test for the fourth Month
housing = flood_df['t+4']
pop = flood_df['t+4p']

stats.median_test(housing, pop)

In [ ]:
#Test for the fifth Month
housing = flood_df['t+5']
pop = flood_df['t+5p']

stats.median_test(housing, pop)

In [ ]:
#Test for the sixth Month
housing = flood_df['t+6']
pop = flood_df['t+6p']

stats.median_test(housing, pop)

In [ ]:
#create dataframe that captures median time series for each disaster and population

median_dict = {}

flood_list = []
hurricane_list = []
fire_list = []
storms_list = []
snow_list = []
ice_list = []
pop_list = []

for i, j in flood_df.loc[:,'t+1':'t+6'].iteritems():
    flood_list.append(j.median())

for i, j in hurricane_df.loc[:,'t+1':'t+6'].iteritems():
    hurricane_list.append(j.median())

for i, j in fire_df.loc[:,'t+1':'t+6'].iteritems():
    fire_list.append(j.median())

for i, j in storms_df.loc[:,'t+1':'t+6'].iteritems():
    storms_list.append(j.median())

for i, j in snow_df.loc[:,'t+1':'t+6'].iteritems():
    snow_list.append(j.median())

for i, j in ice_df.loc[:,'t+1':'t+6'].iteritems():
    ice_list.append(j.median())

for i, j in normalized_clean_df.loc[:,'t+1p':'t+6p'].iteritems():
    pop_list.append(j.median())

median_dict.update({'flood_median' : flood_list,
                    'hurricane_median' : hurricane_list,
                    'fire_median' : fire_list,
                    'storms_median' : storms_list,
                    'snow_median' : snow_list,
                    'ice_median' : ice_list,
                    'pop_median' : pop_list})

median_df = pd.DataFrame(median_dict)

In [ ]:
#add series to median_df that calculates increase in housing value based on normalized base

base = 100000

median_df['flood_cgrowth($)'] = (median_df['flood_median'] + 1).cumprod() * base
median_df['hurricane_cgrowth($)'] = (median_df['hurricane_median'] + 1).cumprod() * base
median_df['fire_cgrowth($)'] = (median_df['fire_median'] + 1).cumprod() * base
median_df['storms_cgrowth($)'] = (median_df['storms_median'] + 1).cumprod() * base
median_df['snow_cgrowth($)'] = (median_df['snow_median'] + 1).cumprod() * base
median_df['ice_cgrowth($)'] = (median_df['ice_median'] + 1).cumprod() * base
median_df['pop_cgrowth($)'] = (median_df['pop_median'] + 1).cumprod() * base

median_df.to_csv('../Cleaned Data/Median_Data.csv')
median_df.head()